# 개요

- Keras
  - 딥러닝을 몇개의 API 사용만으로 손쉽게 구현할 수 잇는 고수준 api 인터페이스
  - Keras는 백엔드 엔진을 지정하여 딥러닝을 수행
    - tensorflow에서 2017 commit 행사에서 공식적으로 Keras를 하위 패키지로 포함
    - tensorflow 2.x 에서 개발방법론 중 고수준 개발 방식으로 지정하여 권장

In [52]:
import tensorflow as tf
from tensorflow import keras
keras.__version__

'2.8.0'

# 난수 고정

- 공통적으로 어떤 엔진을 사용하던 동일한 결과를 얻기 위한 (재현성) 조치사항
- 단 GPU 학습시 시드 고정이 풀리는 문제가 일부 있다고 피드백

In [53]:
# 시드값 준비

RAND_SEED_VALUE = 0

In [54]:
# os레벨

import os
os.environ[ 'PYTHONJASHSEED' ] = str( RAND_SEED_VALUE )

# 파이썬 레벨

import random
random.seed( RAND_SEED_VALUE )

# numpy 레벨

import numpy as np
np.random.seed( RAND_SEED_VALUE )

# 엔진 레벨

# 1.x 시드 고정
# tf.random.seed( RAND_SEED_VALUE )
# 2.x 시드 고정
# tensorflow._api.v2.random.seed

# 케라스 레벨 (생략)

# 데이터 형식 확인

In [55]:
from tensorflow.keras import backend

In [56]:
backend.image_data_format()

# channels_last: NHWC  -> Tensorflow, Keras (백엔드가 텐서플로우인 경우)
# channels_first: NCHW -> pyTorch

'channels_last'

# 데이터 준비

- 케라스에서 데이터를 받아서 처리
- 데이터는 NHWC 형식으로 (4D) 로 구성
  - MNIST => ( ?, 28, 28, 1 )

In [57]:
from tensorflow.keras.datasets import mnist

In [58]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

In [59]:
# 데이터 정규화 작업, one-hot 인코딩 처리

X_train.shape, y_train.shape, X_test.shape, y_test.shape

((60000, 28, 28), (60000,), (10000, 28, 28), (10000,))

In [60]:
# 정답 확인 -> one-hot 처리 필요

np.unique( y_train )

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=uint8)

In [61]:
# 환경변수 획득

PIXEL_H = X_train.shape[ 1 ]  # 28
PIXEL_W = X_train.shape[ 2 ]  # 28
PIXEL   = PIXEL_H + PIXEL_W   # 784
LABEL_NUM = len( np.unique( y_train ) ) # 10
IN_CHANNEL = 1 # 입력채널수 (grayscale)

In [62]:
# 데이터 포맷에 맞춘 shape 변경

if 'channels_last' == backend.image_data_format(): # NHWC

  # ( 6000, 28, 28, 1 )
  
  X_train = X_train.reshape( -1, PIXEL_H, PIXEL_W, IN_CHANNEL )
  X_test  = X_test.reshape(  -1, PIXEL_H, PIXEL_W, IN_CHANNEL )

else:

  # ( 6000, 1, 28, 28 )
  
  X_train = X_train.reshape( -1, IN_CHANNEL, PIXEL_H, PIXEL_W )
  X_test  = X_test.reshape( -1, IN_CHANNEL, PIXEL_H, PIXEL_W )

In [63]:
# 형변환 (0.0 ~ 255.0)

X_train = X_train.astype( 'float32' )
X_test = X_test.astype( 'float32' )

# 정규화 처리 (0.0 ~ 1.0)

X_train = X_train / np.max( X_train )
X_test = X_test / np.max( X_test )

In [64]:
# 정답 -> one-hot 인코딩 처리

y_train = keras.utils.to_categorical( y_train, LABEL_NUM )
y_test = keras.utils.to_categorical( y_test, LABEL_NUM )

y_train.shape, y_test.shape

((60000, 10), (10000, 10))

# 신경망 구성

- model.add( 레이어 )
  - add(): 특정 축을 전체 시퀀스에 결합

In [65]:
from tensorflow.keras import models, layers

In [66]:
# 1. 네트워크 구성 준비, model에 layer를 추가하면서 신경망 구성이 완료

model = models.Sequential()

In [67]:
if 'channels_last' == backend.image_data_format(): 
  # HWC
  input_shape = ( PIXEL_H, PIXEL_W, IN_CHANNEL )

else:
  # CHW
  input_shape = ( IN_CHANNEL, PIXEL_H, PIXEL_W )

In [68]:
# 2. 합성곱 1층

model.add( layers.Conv2D( 
    filters = 32,             # 합성곱층을 통과하고 나서 결과물의 출력 채널 채널 수 1 -> 32
    kernel_size = ( 5, 5),    # 가중치 (w) 를 공용파라미터로 가진 커널의 크기 (정방향 2D 행렬)
    strides = (1, 1),         # 세로, 가로의 이동량 각각 1
    padding = 'same',         # 소문자로 표현 (저수준 - 대문자), same 동일크기, vaild 유효범위 (축소) 
    activation = 'relu',      # 활성화 함수 relu( 입력이 음수면 0, 입력이 양수면 그대로 )
    input_shape = input_shape # 최초 입력 x가 들어오는 자리이므로, 입력 형태를 표현 (최초층이므로 추가)
    ) )

# 3. 풀링 1층

model.add( layers.MaxPool2D(
    pool_size = (2, 2), # 커널 크기를 지정 (정방형)
    strides = None,     # None을 주면, 커널크기와 동일하다
    padding = 'same'    # 동일 크기 -> k, s, p, x 등은 공식이 성립
)  )
model.add( layers.Dropout(0.1) )

# 4. 합성곱 2층

model.add( layers.Conv2D( 32*2, (5, 5), padding = 'same', activation = 'relu' ) )

# 5. 풀링 2층

model.add( layers.MaxPool2D( padding = 'same' ) )
model.add( layers.Dropout(0.1) )

# 6. 전결합층: 특성이 없는 층은 Dense()로 표현

model.add( layers.Flatten() )
model.add( layers.Dense( 1024, activation = 'relu' ) )

# 7. 과적합방지층 (드롭아웃층)

model.add( layers.Dropout(0.1) )

# 8. 출력층

model.add( layers.Dense( LABEL_NUM, activation = 'softmax' ) )


# 학습 및 최적화, 예측 구성

model.compile( 도구 )

In [69]:
model.compile( 
    optimizer = 'rmsprop',                        # 최적화 도구
    loss = keras.losses.categorical_crossentropy, # 손실함수
    metrics = [ 'accuracy' ]                      # 정확도
 )

# 학습

- model.fit(파라미터)

In [70]:
# 학습 관련 변수 설정

EPOCHS =  10     # 10세대 학습을 진행, 전체 데이터플로 학습에 동원
BATCH_SIZE = 128 # 설정값 (앞선 과정에서는 50)

In [71]:
if 0:
  model.fit( 
      x = X_train,                  # 훈련 데이터
      y = y_train,                  # 정답 데이터
      batch_size = BATCH_SIZE,      # 한 번의 학습 시 동원되는 데이터량
      epochs = EPOCHS,              # 세대학습 수 세팅
      verbose = 'auto',             # 로그 출력
      callbacks = None,             # 조기학습 종료 등 콜백함수등록
  )

# 학습 개선 - 조기학습종료 도입

- 특징
  - 학습 성과가 더이상 나오지 않을 때 추가되는 학습은 무의미
  - 변동폭이 더이상 진행되지 않으면 학습을 종료시킨다 (조기학습 종료)
  - 콜백함수 등록 후 처리

In [72]:
from tensorflow.keras.callbacks import EarlyStopping

# 조기 학습 종료 객체

early_stopping = EarlyStopping()

In [73]:
with tf.device('/device:GPU:0'): 
  history = model.fit(
    x=X_train,       # 훈련 데이터
    y=y_train,       # 정답 데이터
    batch_size=BATCH_SIZE, # 한번의 학습시 동원되는 데이터량
    epochs=EPOCHS,   # 세대학습수 세팅 
    #verbose='auto', # 로그 출력
    callbacks=[early_stopping], # 조기학습 종료등 콜백함수등록
    validation_split=0.2 # 검증 데이터 비율    
  )

Epoch 1/10
375/375 [==============================] - 8s 19ms/step - loss: 0.1790 - accuracy: 0.9438 - val_loss: 0.0454 - val_accuracy: 0.9868
Epoch 2/10
375/375 [==============================] - 7s 20ms/step - loss: 0.0460 - accuracy: 0.9863 - val_loss: 0.0423 - val_accuracy: 0.9867


- 조기 학습 종료를 이용하여 2세대해서 학습이 완료되었다

In [74]:
# 이 정보를 이용하여 시각화 가능하다

history.history

{'accuracy': [0.9438124895095825, 0.9862916469573975],
 'loss': [0.17900213599205017, 0.04603469744324684],
 'val_accuracy': [0.9868333339691162, 0.9866666793823242],
 'val_loss': [0.04536870867013931, 0.0422588512301445]}

# 모델 덤프

In [75]:
model.save( 'mnist_cnn.h5' )

# 모델 로드 (서비스측)

In [76]:
from tensorflow.keras.models import load_model

In [77]:
loadedModel = load_model( 'mnist_cnn.h5' )

In [78]:
# 테스트 데이터를 넣어 예측 수행

loadedModel.evaluate( X_test, y_test )

313/313 [==============================] - 2s 5ms/step - loss: 0.0370 - accuracy: 0.9880


[0.03701651096343994, 0.9879999756813049]